### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)


# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Calculate total student
total_schools=school_data["school_name"].count()

#Calculate total schools
total_students= school_data["size"].sum()

#calculate total budget
total_budget = school_data["budget"].sum()

#calculate average math score
average_math_score =student_data["math_score"].mean()

#calculate average reading score
average_reading_score=student_data["reading_score"].mean()

#calculate percent passing math
students_passing_math = student_data.loc[school_data_complete["math_score"] >= 70]
pct_passing_math = students_passing_math["student_name"].count()/total_students

#calculate perent passing reading
students_passing_reading = student_data.loc[school_data_complete["reading_score"] >= 70]
pct_passing_reading = students_passing_reading["student_name"].count()/total_students

# Store the results in data frame district_summary_df 
district_summary=pd.DataFrame({"Total Schools":[total_schools],
                                  "Total Students": [total_students],
                                  "Total Budget": [total_budget], 
                                  "Average Math Score": [average_math_score], 
                                  "Average Reading Score": [average_reading_score],
                                  "% Passing Math": [100*pct_passing_math],
                                  "% Passing Reading": [100*pct_passing_reading],
                                  "% Overall Passing Rate": [(average_math_score+average_reading_score)/2]})
# Use Map to format the columns
district_summary["Total Students"]=district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"]=district_summary["Total Budget"].map("${:,.2f}".format)

district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [7]:
school_df_1 = school_data_complete.groupby('school_name').agg(
            total_students=pd.NamedAgg(column='student_name', aggfunc = 'count'),
            avg_math_score = pd.NamedAgg(column='math_score', aggfunc = 'mean'),
            avg_reading_score = pd.NamedAgg(column ='reading_score', aggfunc='mean'))
school_df_1

passing_math_df= school_data_complete.loc[school_data_complete['math_score'] >=70]
passing_reading_df= school_data_complete.loc[school_data_complete['reading_score'] >=70]

school_df_2 = passing_math_df.groupby('school_name').agg(
             total_passing_math = pd.NamedAgg(column='student_name', aggfunc= 'count'))
#school_df_2

school_df_3 = passing_reading_df.groupby('school_name').agg(
             total_passing_reading = pd.NamedAgg(column = 'student_name', aggfunc = 'count'))
#school_df_3
school_df_4 = school_data[['school_name','type','budget', 'size']]

# school_df_4

school_summary = pd.merge(school_df_1,  school_df_2, on='school_name')
school_summary = pd.merge(school_summary, school_df_3, on='school_name')
school_summary = pd.merge(school_summary, school_df_4, on='school_name')

school_summary["pct_passing_math"]= 100*school_summary["total_passing_math"]/school_summary["total_students"]
school_summary['pct_passing_reading'] = 100*school_summary["total_passing_reading"]/school_summary["total_students"]
school_summary['overall_passing_rate'] = (school_summary['pct_passing_math']+ \
                                           school_summary['pct_passing_reading'])/2
school_summary['per_student_budget'] = school_summary['budget']/school_summary['size']

#Reorganize using the double brackets

school_summary = school_summary[['school_name','type', 'total_students',
                            'budget', 'per_student_budget',
                              'avg_math_score', 'avg_reading_score',
                            'pct_passing_math', 'pct_passing_reading',
                             'overall_passing_rate']]

school_summary = school_summary.rename(columns={"type":"School Type",
                                                 "total_students": "Total Students",
                                                 "budget": "Total School Budget",
                                                 "per_student_budget": "Per Student Budget",
                                                 "avg_math_score": "Average Math Score",
                                                 "avg_reading_score": "Average Reading Score",
                                                 "pct_passing_math": "% Passing Math",
                                                 "pct_passing_reading": "% Passing Reading",
                                                  "% overall_passing_rate": "Overall Passing Rate"})
# format columns with map function

school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary = school_summary.set_index("school_name")

#### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [8]:
top_5_schools = school_summary.sort_values("overall_passing_rate", ascending=False)
top_5_schools.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,overall_passing_rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


# Bottom Performing Schools (By Passing Rate)


* Sort and display the five worst-performing schools

In [9]:
bottom_5_schools = school_summary.sort_values("overall_passing_rate")
bottom_5_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,overall_passing_rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

### * Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
nine_grade = school_data_complete.loc[school_data_complete["grade"] =="9th"]
ten_grade = school_data_complete.loc[school_data_complete["grade"] =="10th"]
eleven_grade = school_data_complete.loc[school_data_complete["grade"] =="11th"]
twelve_grade = school_data_complete.loc[school_data_complete["grade"] =="12th"]


avg_9_grades  = nine_grade.groupby('school_name').agg(
             grade_9_avg = pd.NamedAgg(column='math_score', aggfunc= 'mean'))
avg_10_grades = ten_grade.groupby('school_name').agg(
             grade_10_avg = pd.NamedAgg(column='math_score', aggfunc= 'mean'))
avg_11_grades = eleven_grade.groupby('school_name').agg(
             grade_11_avg = pd.NamedAgg(column='math_score', aggfunc= 'mean'))
avg_12_grades = twelve_grade.groupby('school_name').agg(
             grade_12_avg = pd.NamedAgg(column='math_score', aggfunc= 'mean'))

math_scores_by_grade = pd.merge(avg_9_grades, avg_10_grades, on="school_name")
math_scores_by_grade = pd.merge(math_scores_by_grade, avg_11_grades, on="school_name")
math_scores_by_grade = pd.merge(math_scores_by_grade, avg_12_grades, on="school_name")

math_scores_by_grade


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:

avg_9_grades = nine_grade.groupby('school_name').agg(
             grade_9_avg = pd.NamedAgg(column='reading_score', aggfunc= 'mean'))
avg_10_grades = ten_grade.groupby('school_name').agg(
             grade_10_avg = pd.NamedAgg(column='reading_score', aggfunc= 'mean'))
avg_11_grades = eleven_grade.groupby('school_name').agg(
             grade_11_avg = pd.NamedAgg(column='reading_score', aggfunc= 'mean'))
avg_12_grades = twelve_grade.groupby('school_name').agg(
             grade_12_avg = pd.NamedAgg(column='reading_score', aggfunc= 'mean'))

reading_scores_by_grade = pd.merge(avg_9_grades, avg_10_grades, on="school_name")
reading_scores_by_grade = pd.merge(reading_scores_by_grade, avg_11_grades, on="school_name")
reading_scores_by_grade = pd.merge(reading_scores_by_grade, avg_12_grades, on="school_name")

reading_scores_by_grade

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [ ]:
school_summary["spending_range_by_student"]= pd.cut(school_summary["per_student_budget"], 
                                             spending_bins, labels=group_names)
school_summary.set_index("spending_range_by_student")
#school_summary.drop["school_name", "type","spending_per_studeint"]
school_summary= school_summary.groupby("spending_range_by_student")
school_summary.max()


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_summary["student_range_by_school"]= pd.cut(school_summary["total_students"], 
                                             size_bins, labels=group_names)
school_summary.set_index("student_range_by_school")
school_summary=school_summary.groupby("student_range_by_school")
school_summary.max()


## Scores by School Type

* Perform the same operations as above, based on school type.